In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return your GPU model


True
NVIDIA GeForce RTX 3050


In [2]:
#download the model and tokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")

print("Model and tokenizer downloaded successfully!")


h:\Coding Project\MindEase\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
h:\Coding Project\MindEase\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kumar\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activa

Model and tokenizer downloaded successfully!


In [4]:
#donwload the data

import pandas as pd
import glob

# Define the path to all CSV files in the "labelled data" folder
csv_files = glob.glob("data/reddit-mental-health-dataset/Original Reddit Data/Labelled Data/*.csv")

# Read all CSVs and combine them
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Show basic dataset info
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823 entries, 0 to 822
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   score      800 non-null    float64
 1   selftext   800 non-null    object 
 2   subreddit  800 non-null    object 
 3   title      800 non-null    object 
 4   Label      800 non-null    object 
 5   CAT 1      200 non-null    object 
dtypes: float64(1), object(5)
memory usage: 38.7+ KB
None
   score                                           selftext subreddit  \
0    1.0  Tried to watch this documentary “anxious Ameri...   Anxiety   
1    1.0  i’m currently laying in bed wide awake, feelin...   Anxiety   
2    2.0  Second time trying weed. First time felt close...   Anxiety   
3    1.0  I am not posting this for me, but rather for m...   Anxiety   
4    1.0  21 year old male been dealing with anxiety eve...   Anxiety   

                                               title             Label CAT 1  


After downloading the data move the data to "data" folder and run the below code to get the data in the required format.

In [5]:
import pandas as pd
import glob

# Load all CSV files from the correct path
csv_files = glob.glob("data/reddit-mental-health-dataset/Original Reddit Data/Labelled Data/*.csv")

# Read and merge all CSVs
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Keep only relevant columns
df = df[['title', 'selftext', 'Label']]

# Drop missing values in 'selftext'
df = df.dropna(subset=['selftext'])

# Combine title and selftext into one column
df['text'] = df['title'].fillna('') + " " + df['selftext']

# Keep only the final processed text and label
df = df[['text', 'Label']]

# Show updated dataset info
print(df.info())
print(df.head())

# Save the cleaned dataset as a CSV file
df.to_csv("data/reddit-mental-health-dataset/cleaned_mental_health_data.csv", index=False)

print("✅ Cleaned dataset saved successfully!")


<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 0 to 822
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    800 non-null    object
 1   Label   800 non-null    object
dtypes: object(2)
memory usage: 18.8+ KB
None
                                                text             Label
0  Do people get over anxiety? Tried to watch thi...  Drug and Alcohol
1  does anyone else have this big fear of suddenl...  Drug and Alcohol
2  3 hour long panic attack after trying weed Sec...  Drug and Alcohol
3  Please leave in the comments ANYTHING that has...  Drug and Alcohol
4  Alcohol induced 21 year old male been dealing ...  Drug and Alcohol
✅ Cleaned dataset saved successfully!


In [6]:
from transformers import AutoTokenizer
import pandas as pd

# Load cleaned dataset
df = pd.read_csv("data/reddit-mental-health-dataset/cleaned_mental_health_data.csv")

# Load tokenizer for TinyLlama
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_text(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=512)

# Apply tokenization
df["input_ids"] = df["text"].apply(lambda x: tokenize_text(str(x))["input_ids"])

# Keep only tokenized inputs and labels
df = df[["input_ids", "Label"]]

# Save tokenized data as Parquet for efficient processing
df.to_parquet("data/reddit-mental-health-dataset/tokenized_mental_health_data.parquet", engine="pyarrow")

print("✅ Tokenization complete! Data saved as Parquet.")


✅ Tokenization complete! Data saved as Parquet.


In [7]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from datasets import load_dataset
import torch
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig

# Enable 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Use 8-bit quantization
    llm_int8_enable_fp32_cpu_offload=True  # Offload CPU computations to save VRAM
)

# Load model and force it onto the current CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={'': torch.cuda.current_device()},  # Assign model to GPU
    quantization_config=quantization_config  # Apply quantization
)
  # Move model to GPU




# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05, 
    bias="none"
)
model = get_peft_model(model, lora_config)

# Load tokenized dataset
from datasets import DatasetDict

# Load tokenized dataset
dataset = load_dataset("parquet", data_files={"data": "data/reddit-mental-health-dataset/tokenized_mental_health_data.parquet"})

# Convert dataset to dictionary format
dataset = dataset["data"].train_test_split(test_size=0.1)

# Ensure `labels` are the same as `input_ids`
dataset = DatasetDict({
    "train": dataset["train"].map(lambda x: {"labels": x["input_ids"]}),
    "eval": dataset["test"].map(lambda x: {"labels": x["input_ids"]})
})


# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_results",
    per_device_train_batch_size=2,  # Lower batch size for 4GB GPU
    per_device_eval_batch_size=2,
    num_train_epochs=3,  # Train for 3 epochs
    save_total_limit=2,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["eval"]  # Add evaluation dataset
)


# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
print("✅ Fine-tuning complete! Model saved.")


Generating data split: 800 examples [00:00, 63402.36 examples/s]
Map: 100%|██████████| 80/80 [00:00<00:00, 1386.03 examples/s]


Epoch,Training Loss,Validation Loss
1,1.460400,No log
2,1.404700,No log
3,1.381500,No log


✅ Fine-tuning complete! Model saved.


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from peft import PeftModel
import torch

# Define paths
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Base model
FINETUNED_PATH = "./fine_tuned_model"  # Where fine-tuned model will be saved

# Load base model
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="cpu")

# Load LoRA adapters and merge them
model = PeftModel.from_pretrained(model, FINETUNED_PATH)
model = model.merge_and_unload()  # Merge LoRA weights

# Save the full fine-tuned model
model.save_pretrained(FINETUNED_PATH)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.save_pretrained(FINETUNED_PATH)

# Save configuration
config = AutoConfig.from_pretrained(BASE_MODEL)
config.save_pretrained(FINETUNED_PATH)

print("✅ Full fine-tuned model saved successfully!")


✅ Full fine-tuned model saved successfully!


In [11]:
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Define model path
MODEL_PATH = "./fine_tuned_model"

# Ensure model is properly loaded
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Function to generate responses
def chat_with_model(prompt, max_length=150):
    formatted_prompt = f"User: {prompt}\nAI:"
    
    input_ids = tokenizer(formatted_prompt, return_tensors="pt").input_ids.to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response.replace(formatted_prompt, "").strip()
    return response

# Test the model with sample inputs
sample_inputs = [
    "I have been feeling very anxious lately. What should I do?",
    "How can I manage my stress effectively?",
    "I feel lonely and isolated. Can you help?",
]

# Generate responses for each input
for query in sample_inputs:
    response = chat_with_model(query)
    print(f"🗣️ **User:** {query}\n🤖 **AI:** {response}\n")


h:\Coding Project\MindEase\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🗣️ **User:** I have been feeling very anxious lately. What should I do?
🤖 **AI:** Please try to avoid any negative stimuli and try to stay in a positive environment as much as possible. It's important that you are present in the moment and focus on your breathing or some other activity that helps reduce anxiety symptoms. Avoid overthinking things, it just adds more pressure for yourself to feel anxious. Practice mindfulness exercises like meditation, yoga, or deep breathing techniques when feeling anxious. And finally, seek help from someone who can provide guidance or support if needed. Remember that everyone has their own unique struggles and feelings, so don’t be afraid of seeking professional advice

🗣️ **User:** How can I manage my stress effectively?
🤖 **AI:** It's common to feel stressed at times, but it is essential to understand that there are ways to manage your stress better. Here are some tips:

1. Practice mindfulness - Start by taking a few minutes each day to focus on yo

In [2]:
from huggingface_hub import HfApi

# Replace with your Hugging Face username and model name
USERNAME = "tezodipta"
MODEL_NAME = "MindEase-Assistant-v0.1"

api = HfApi()

# Create a repository on Hugging Face (skip if already created)
api.create_repo(repo_id=f"{USERNAME}/{MODEL_NAME}", private=False, exist_ok=True)

# Upload the entire **fine-tuned model folder** instead of a sharded model
api.upload_folder(
    folder_path="./fine_tuned_model",  # Make sure this folder contains model files
    repo_id=f"{USERNAME}/{MODEL_NAME}",
)

print(f"✅ Fine-Tuned Model uploaded successfully: https://huggingface.co/{USERNAME}/{MODEL_NAME}")


adapter_model.safetensors:   0%|          | 0.00/4.52M [00:00<?, ?B/s]


adapter_model.safetensors:   2%|▏         | 98.3k/4.52M [00:00<00:04, 935kB/s]

tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 527kB/s] 0, 22.9MB/s]
adapter_model.safetensors: 100%|██████████| 4.52M/4.52M [00:01<00:00, 2.45MB/s]

model.safetensors: 100%|██████████| 4.40G/4.40G [03:26<00:00, 21.3MB/s]

Upload 3 LFS files: 100%|██████████| 3/3 [03:27<00:00, 69.13s/it] 


✅ Fine-Tuned Model uploaded successfully: https://huggingface.co/tezodipta/MindEase-Assistant-v0.1


In [4]:
#accessing a model from huggingface hub using api call
from huggingface_hub import InferenceClient

# Fetch the API key from a file named api_key.txt
with open("api.txt", "r") as file:
	api_key = file.read().strip()

client = InferenceClient(
	provider="together",
	api_key=api_key
)

messages = [
	{
		"role": "user",
		"content": "What is the capital of France?"
	}
]

completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.3", 
	messages=messages, 
	max_tokens=500,
)

print(completion.choices[0].message)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: Root=1-67b42900-6d2a06a35b2b42a70a92e0fd;9ee54482-c3b7-40f2-8a7c-c308fa50d0c1)

You have exceeded your monthly included credits for Inference Endpoints. Subscribe to PRO to get 20x more monthly allowance.

In [7]:
!pip install together

  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.6 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.6 MB 1.3 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.6 MB 1.3 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.6 MB 1.3 MB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.6 MB 1.4 MB/s eta 0:00:01
   --------------------------- ------------ 1.8/2.6 MB 1.4 MB/s eta 0:00:01
   ------------------------------- -------- 2.1/2.6 MB 1.4 MB/s eta 0:00:01
   ----------------------------------- ---- 2.4/2.6 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 1.3 MB/s eta

In [27]:
from together import Together

with open("together_key.txt", "r") as file:
	api_key = file.read().strip()
# Initialize the client with API key
client = Together(api_key=api_key)

response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
    max_tokens=200,
)

print(response.choices[0].message.content)


The city that never sleeps! New York has endless options for entertainment, culture, and adventure. Here are some fun things to do in New York:

**Iconic Landmarks:**

1. Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.
2. Central Park: Explore the park's many walking paths, lakes, and landmarks like the Bethesda Fountain and Loeb Boathouse.
3. Times Square: Experience the bright lights and energy of the "Crossroads of the World."
4. Empire State Building: Enjoy panoramic views of the city from the observation deck on the 86th floor.
5. Brooklyn Bridge: Walk or bike across the iconic bridge for spectacular city views.

**Museums and Galleries:**

1. The Metropolitan Museum of Art: One of the world's largest and most renowned museums, with a collection that spans over 5,000 years of human history.
2.


In [18]:
!pip install groq

In [29]:
#gorq api

from groq import Groq

with open("gorq_key.txt", "r") as file:
	api_key = file.read().strip()

client = Groq(api_key=api_key)
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": "hello"
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?

In [ ]:
#open router api
from openai import OpenAI
with open("openrouter_key.txt", "r") as file:
	api_key = file.read().strip()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

completion = client.chat.completions.create(
  extra_body={},
  model="mistralai/mistral-saba",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ],
  max_tokens=100,
)
print(completion.choices[0].message.content)

The question "What is the meaning of life?" is one of the most profound and enduring philosophical inquiries. Different cultures, religions, and philosophical traditions offer a variety of answers. Here are a few perspectives:

1. **Existentialism**: Existentialists like Jean-Paul Sartre argued that life has no inherent meaning, and it is up to each individual to create their own purpose.

2. **Religious and Spiritual Views**: Many religions provide their own interpretations. For example:
   - In Christianity
